In [1]:
#packages
import pandas as pd
import pickle
import csv
import os
import re
import emoji
from textblob import TextBlob
import nltk
from langdetect import detect
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix
import gensim
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import coo_matrix, hstack
from scipy.sparse import csr_matrix
import sklearn
from scipy import spatial

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
#load all dictionary to add precleaned txt, prepare for manual verification of cluster result
def read_json_to_dataframe(filename):
    print(filename)
    #f = open(filename,encoding = 'utf-16').read()
    f1 = pd.read_json(filename, orient='records',lines=True,encoding = 'utf-16')
    return f1
def read_in(path):
    files = os.listdir(path)
    all_dfs = []
    print(files)
    for i in files:
        if 'json' in i:
            each_df = read_json_to_dataframe(path+i)
            all_dfs.append(each_df)
            
    return pd.concat(all_dfs, ignore_index=True)

all_df = read_in('C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/')

['02-17-2021.json', '02-18-2021.json', '02-19-2021.json', '02-20-2021.json', '02-21-2021.json', '02-22-2021.json', '02-23-2021.json', '02-24-2021.json']
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-17-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-18-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-19-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-20-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-21-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-22-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-23-2021.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/newdata/02-24-2021.json


In [3]:
all_df

,og_tweet_txt,og_tweet_time,og_tweet_id,og_tweet_user_id,og_tweet_user_name,og_tweet_user_desc,og_tweet_user_vrifd,og_tweet_user_loc,tweet_txt,user_loc,...,tweet_id,user_id,user_name,tweet_likes,tweet_source,hashtags,user_acc_cr_time,user_verified,user_total_tweets,user_followers
0,#Gates continues to move the goalposts. Meanwh...,2021-02-17 21:22:30,1362150453610483713,337808606,RobertKennedyJr,"Robert F. Kennedy, Jr. is founder + chairman o...",True,"Los Angles, California",RT @RobertKennedyJr: #Gates continues to move ...,Nowhere yet everywhere,...,1362255911662723079,1314666986878595072,Nick29433777,0,Twitter for iPhone,Gates,2020-10-09 20:40:55,False,36398,77
1,Former Detroit TV Anchor Dies One Day After Ta...,2021-02-18 03:40:14,1362245512859123713,35013035,BusyDrT,Doctor who speaks out about vaccines.,False,"Cleveland, Ohio",RT @BusyDrT: Former Detroit TV Anchor Dies One...,Now,...,1362255915198521345,1151512982289035265,JimSatone,0,Twitter Web App,,2019-07-17 15:24:23,False,20125,669
2,Here’s a video of Ron DeSantis being a big bab...,2021-02-17 17:54:20,1362098065612541957,1354488811305259121,RemoveRon,Dedicated to taking down Ron DeSantis in 2022....,False,Florida,RT @RemoveRon: Here’s a video of Ron DeSantis ...,"New Jersey, USA",...,1362255916624666625,906531427,tb_terry,0,Twitter for iPhone,,2012-10-26 18:28:07,False,63017,4607
3,,NaT,,,,,,,@JesseBWatters Is anyone gonna mention Biden f...,On a Lake,...,1362255917085986817,34742700,tinberry,0,Twitter Web App,,2009-04-23 21:51:32,False,1737,21
4,He literally had the COVID-19 vaccine in his b...,2021-02-17 02:59:05,1361872770255224832,239005322,CalebJHull,"Monetization, Content, & Viral Video // 🚀 Ball...",True,Dallas // DC // NYC,RT @CalebJHull: He literally had the COVID-19 ...,Montreal,...,1362255920110178306,93194555,alexsimonelis,0,Twitter for Android,,2009-11-28 14:36:55,False,61621,673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970703,They’re making you get tested for Covid before...,2021-02-25 01:41:54,1364752447831375873,1195136746524012544,TrishForTrump,Real Estate Broker...Investor...USA ...”Joe’s ...,False,"California, USA",RT @TrishForTrump: They’re making you get test...,"Cyberdyne Systems, Inc.",...,1364847442609336325,2590012027,Algerwins,0,Twitter Web App,,2014-06-26 17:42:54,False,66110,5216
970704,Whose coming 🥳🥳🥳🥳\n\nhttps://t.co/nPXuNOQFch,2021-02-24 22:19:00,1364701389692280835,83197153,Lyndonx,This is real and raw. Sharing experiences & li...,True,Zapped it,RT @Lyndonx: Whose coming 🥳🥳🥳🥳\n\nhttps://t.co...,London,...,1364847445151219714,3003029949,ThemysciranJZ,0,Twitter Web App,,2015-01-28 12:10:27,False,19948,1262
970705,Tremendous flexibility of synthetic biology: M...,2021-02-25 01:48:52,1364754203663564800,282076470,AgBioWorld,"Professor, biotech guru. I am curious about sc...",False,"Alabama, USA",RT @AgBioWorld: Tremendous flexibility of synt...,"Cold Spring Harbor, NY",...,1364847445717504001,1692248610,JasonWilliamsNY,0,Twitter for iPhone,,2013-08-22 22:53:04,False,75992,4314
970706,"The #Philippines will receive a batch of 600,0...",2021-02-25 06:02:00,1364817905364340739,1115874631,CGTNOfficial,CGTN is an international media organization. I...,True,"Beijing, China",RT @CGTNOfficial: The #Philippines will receiv...,Republic of Korea,...,1364847445738356738,890916095963430912,Totoyami1,0,Twitter for Android,Philippines,2017-07-28 12:45:24,False,5696,8
